In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
import numpy as np

# Description Data

In [2]:
data = pd.read_csv('./description.csv')
data

,username,summary,storyline
0,47metersdown,Four teen girls diving in a ruined underwater ...,47 Meters Down: Uncaged follows the diving adv...
1,adogsjourneymovie,A dog finds the meaning of his own existence t...,A dog finds the meaning of his own existence t...
2,aftermathmovie,"Post World War II, a British colonel and his w...","Set in postwar Germany in 1946, Rachael Morgan..."
3,aftermovie,A young woman falls for a guy with a dark secr...,"Based on Anna Todd's novel, AFTER follows Tess..."
4,alitamovie,"A deactivated cyborg is revived, but cannot re...",Alita is a creation from an age of despair. Fo...
...,...,...,...
106,whatmenwant,A woman is boxed out by the male sports agents...,A woman is boxed out by the male sports agents...
107,wonderparkmovie,Wonder Park tells the story of an amusement pa...,"June, an optimistic, imaginative girl, discove..."
108,xmenmovies,Jean Grey begins to develop incredible powers ...,The X-Men. Protectors of peace. Jean Grey is o...
109,yardiefilm,British crime drama film directed by Idris Elb...,"Set in '70s Kingston and '80s Hackney, Yardie ..."


# Sentence remove stopword.... and do some processing

In [3]:
#還沒移除標點符號
def sentence_processing(sentence):
    #stopword get
    stop_words = set(stopwords.words('english')) 
    filtered_sentence = []
    #delete no-English part 
    if sentence != 'None':
        sentence =re.sub('[^a-zA-Z]',' ',sentence)
        #tokenization
        word_tokens = word_tokenize(sentence) 
        """
        #stemming
        ps = PorterStemmer()
        stemed_words = []
        for work_token in word_tokens:
            stemed_words.append(ps.stem(work_token))
        """
        #filtered_sentence = [w for w in stemed_words if not w in stop_words] #有stem的狀態下
        filtered_sentence = [w.lower() for w in word_tokens if not w in stop_words] #沒有stem的狀態下
        #print(stemed_words)
        #print(word_tokens) 
        #print(filtered_sentence)
    return filtered_sentence

In [4]:
#Each description do tokenization, stopword removes, stemming to make a word list for each youtuber 
doc_list = []
for d in data['summary']:
    filtered_sentence = sentence_processing(d)
    doc_list.append(filtered_sentence)
len(doc_list)

111

# Word2Vec

word2vec 是 Google 的一個開源工具，能夠根據輸入的「詞的集合」計算出詞與詞之間的距離
它將「字詞」轉換成「向量」形式，可以把對文本內容的處理簡化為向量空間中的向量運算，計算出向量空間上的相似度，來表示文本語義上的相似度

相關資料 https://code.google.com/archive/p/word2vec/     

github  https://github.com/dav/word2vec

How to download pre-trained data(Google News dataset) https://mccormickml.com/2016/04/12/googles-pretrained-word2vec-model-in-python/

How to implement word2Vec https://towardsdatascience.com/using-word2vec-to-analyze-news-headlines-and-predict-article-success-cdeda5f14751

In [5]:
import gensim

# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('./model/GoogleNews-vectors-negative300.bin', binary=True)
# Check dimension of word vectors(Each word is represented as a feature vector of 300 dimensions)
model.vector_size

300

In [6]:
index = 0
for doc in doc_list:
    # Filter the list of vectors to include only those that Word2Vec has a vector for
    vector_list = [model[word] for word in doc if word in model.vocab]

    # Create a list of the words corresponding to these vectors
    words_filtered = [word for word in doc if word in model.vocab]

    # Zip the words together with their vector representations
    word_vec_zip = zip(words_filtered, vector_list)

    # Cast to a dict so we can turn it into a DataFrame
    word_vec_dict = dict(word_vec_zip)
    df = pd.DataFrame.from_dict(word_vec_dict, orient='index')
    df.to_csv('./feature matrix/'+data['username'][index]+'.csv')
    index += 1

# CNN in keras with pretrained word2vec weights
https://www.kaggle.com/marijakekic/cnn-in-keras-with-pretrained-word2vec-weights/notebook

In [9]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

word_vectors = KeyedVectors.load_word2vec_format('./model/GoogleNews-vectors-negative300.bin', binary=True)

EMBEDDING_DIM=300
vocabulary_size=min(len(word_index)+1,NUM_WORDS)
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=NUM_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del(word_vectors)

from keras.layers import Embedding
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True)

NameError: name 'word_index' is not defined